In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    ['https://rag-search-using-elastic.es.us-central1.gcp.cloud.es.io'],
    basic_auth=('elastic', 'y0Wjxx5nEPyPyt8l07pDaTns'),
    verify_certs=False
    #ca_certs="A2525B64D8BFD084D946539261844AC9A3F7DBDC.crt"
)



c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\elasticsearch\_sync\client\__init__.py:397: SecurityWarning: Connecting to 'https://rag-search-using-elastic.es.us-central1.gcp.cloud.es.io:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [ ]:
index_name = 'recipes'
mapping = {
    "mappings":
           {
               'properties': {
                "group": { "type": "text" },
                "name": { "type": "text" },
                'rating': { 'type': 'text' },
                'n_rater': { 'type': 'text' },
                'n_reviewer': { 'type': 'text' },
                    'summary': {
                        'type': 'text',
                        'analyzer': 'english'
                    },
                'process': { 'type': 'text' },
                'ingredient': {
                        'type': 'text',
                },
                'ml.tokens': {
                    'type': 'rank_features'
                }
            }
        }
    }
es.indices.create(index=index_name, body=mapping)

In [1]:
import pandas as pd
with open('allrecipes.csv', 'r', encoding='utf-8', errors='ignore') as file:
    df = pd.read_csv(file)

In [2]:
recipes = df.to_dict('records')
print(f"Number of documents: {len(recipes)}")

Number of documents: 6204


In [3]:
recipes

[{'group': 'breakfast-and-brunch.eggs.breakfast-burritos',
  'name': 'Ham and Cheese Breakfast Tortillas',
  'rating': 0.0,
  'n_rater': 0,
  'n_reviewer': 44,
  'summary': 'This is great for a special brunch or even a quick and easy dinner. Other breakfast meats can be used, but the deli ham is the easiest since it is already fully cooked.',
  'process': 'prep: 30 mins,total: 30 mins,Servings: 4,Yield: 4 servings',
  'ingredient': '12 eggs + <U+2153> cup milk + 3 slices cooked ham, diced + 2 green onions, minced + salt and pepper to taste + 4 ounces Cheddar cheese, shredded + 4 (10 inch) flour tortillas +  cup salsa'},
 {'group': 'breakfast-and-brunch.eggs.breakfast-burritos',
  'name': 'Freeze-and-Reheat Breakfast Burritos',
  'rating': 0.0,
  'n_rater': 0,
  'n_reviewer': 16,
  'summary': 'This recipe for hearty breakfast burritos is intended to be made in a large batch so that individual burritos can be frozen and reheated as needed. Making these in advance and freezing is a big ti

# Before running the next steps set up model and run elser pipeline. Check readme.txt for setting up in Dev Console

# Creating a bulk indexing sequence

In [23]:
import math
bulk_index_body = []
for index, recipe in enumerate(recipes):
    document = {
        "_index": "recipes",
        "pipeline": "elser-v2-recipes",
        "_source": recipe
    }
    if math.isnan(recipe['rating']):        
        recipe['rating'] = 0.0
    bulk_index_body.append(document)

# perform bulkindex operation

In [21]:
len(bulk_index_body)

6204

# Bulk Index Data to Elastic

In [26]:
from elasticsearch import helpers
from elasticsearch.helpers import BulkIndexError

try:
    response = helpers.bulk(es, bulk_index_body, chunk_size=500, request_timeout=60*3)
    print ("\nRESPONSE:", response)
except BulkIndexError as e:
    i=0
    for error in e.errors:
        i=i+1
        print(f"Document ID: {error['index']['_id']}")
        print(f"Error Type: {error['index']['error']['type']}")
        print(f"Error Reason: {error['index']['error']['reason']}")

print(i)

C:\Users\navin.v.i.ASSYST-COC\AppData\Local\Temp\ipykernel_8624\54291536.py:5: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = helpers.bulk(es, bulk_index_body, chunk_size=500, request_timeout=60*3)
c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rag-search-using-elastic.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rag-search-using-elastic.es.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://url


RESPONSE: (6204, [])
500


# Building a recipe generator
# In the context of our recipe retrieval task, our goal is to maximize the relevance of the returned recipes based on a user’s query. We will utilize a combination of classic full-text search (via BM25), semantic search (with ELSER), and a robust rank fusion method (RRF). This combination allows us to handle more complex queries and return results that align closely with the user’s intent.

# Initiate the generator using ## Streamlit Application